# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create the path to open the file
weatherpy_data = "../WeatherPy/cities.csv"

In [3]:
# Read the file
cities_temperature = pd.read_csv(weatherpy_data)

# Organize file headers
cities_temperature = cities_temperature[["City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
cities_temperature.dropna(inplace = True)
cities_temperature.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,new norfolk,-42.78,147.06,61.00,65,0,3.00,AU,1602036388
1,kaniama,-7.57,24.18,68.50,79,4,1.83,CD,1602036390
2,vestmannaeyjar,63.44,-20.27,43.61,80,100,2.93,IS,1602036392
3,pacific grove,36.62,-121.92,63.00,87,40,3.67,US,1602036229
4,barentsburg,43.32,145.57,62.60,59,20,8.05,JP,1602036399


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = cities_temperature[["Lat", "Lng"]]

# Store humidity values into a series to use it as weight 
humidity = cities_temperature["Humidity"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")
fig = gmaps.figure(center=(0, 0), zoom_level=2.25)
# Create a heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure 
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow the data frame
narrow_cities_data = cities_temperature.loc[((cities_temperature["Max Temp"]<=85.00) & (cities_temperature["Max Temp"]>73.00) & (cities_temperature["Wind Speed"]<10.00) & (cities_temperature["Cloudiness"]==0))]
narrow_cities_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,padang,-15.86,-38.88,75.20,83,0,5.82,BR,1602036492
55,chokurdakh,31.26,32.28,78.80,78,0,6.93,EG,1602036515
80,tiznit,0.49,42.79,74.79,89,0,3.62,SO,1602036573
111,obo,-21.64,-41.05,75.02,90,0,1.16,BR,1602036550
128,mys shmidta,-10.41,-36.43,74.03,82,0,7.72,BR,1602036682
142,port alfred,22.57,59.53,77.74,54,0,5.79,OM,1602036712
143,sabang,21.43,39.83,78.57,70,0,1.25,SA,1602036714
205,dunedin,14.90,-24.50,78.60,73,0,4.32,CV,1602036863
208,samusu,24.69,46.72,77.00,25,0,5.82,SA,1602036694
277,narsaq,27.15,57.08,80.60,83,0,2.24,IR,1602037028


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
#Store into variable named hotel_df
hotel_df = narrow_cities_data[["City", "Lat", "Lng", "Humidity", "Wind Speed", "Country"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

/Users/manuelamunoz/opt/anaconda3/envs/PythonData2/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Lat,Lng,Humidity,Wind Speed,Country,Hotel Name
45,padang,-15.86,-38.88,83,5.82,BR,
55,chokurdakh,31.26,32.28,78,6.93,EG,
80,tiznit,0.49,42.79,89,3.62,SO,
111,obo,-21.64,-41.05,90,1.16,BR,
128,mys shmidta,-10.41,-36.43,82,7.72,BR,
142,port alfred,22.57,59.53,54,5.79,OM,
143,sabang,21.43,39.83,70,1.25,SA,
205,dunedin,14.90,-24.50,73,4.32,CV,
208,samusu,24.69,46.72,25,5.82,SA,
277,narsaq,27.15,57.08,83,2.24,IR,


In [12]:
# Base_url using nearbysearch

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Define params

params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    hotel_name = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"City {index} was not found")

/Users/manuelamunoz/opt/anaconda3/envs/PythonData2/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


City 80 was not found


In [13]:
# Drop the city not found, sort by city name and reset index
hotel_df.sort_values(["City"], inplace=True)
hotel_df.reset_index(inplace =True)
hotel_df

/Users/manuelamunoz/opt/anaconda3/envs/PythonData2/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,City,Lat,Lng,Humidity,Wind Speed,Country,Hotel Name
0,55,chokurdakh,31.26,32.28,78,6.93,EG,فندق رستا بورسعيد Resta Port Said Hotel
1,205,dunedin,14.90,-24.50,73,4.32,CV,Hotel Ocean View & Restaurante Seafood
2,513,genhe,-15.97,-54.97,24,3.83,BR,Taba Hotel
3,463,meyungs,-11.09,-43.14,48,5.88,BR,Hotel Barra Palace
4,128,mys shmidta,-10.41,-36.43,82,7.72,BR,Pousada Raizes
5,277,narsaq,27.15,57.08,83,2.24,IR,Minab Tourism Hotel
6,111,obo,-21.64,-41.05,90,1.16,BR,POUSADA SOBRE AS ÁGUAS
7,45,padang,-15.86,-38.88,83,5.82,BR,Pousada Belmonte
8,142,port alfred,22.57,59.53,54,5.79,OM,Sur Plaza Hotel
9,143,sabang,21.43,39.83,70,1.25,SA,فندق زمازم - Zamazim Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [15]:
fig = gmaps.figure(map_type="TERRAIN")

# Add marker layer ontop of heat map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))